In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
x = torch.tensor([[1, -1], [-1, 1]])
print(x)

tensor([[ 1, -1],
        [-1,  1]])


In [3]:
np_arr = np.array([[1, -1], [-1, 1]])
x = torch.from_numpy(np_arr)
print(x)

tensor([[ 1, -1],
        [-1,  1]])


In [4]:
df = pd.DataFrame([[1, -1], [-1, 1]])
display(df)
x = torch.tensor(df.values)
print(x)

,0,1
0,1,-1
1,-1,1


tensor([[ 1, -1],
        [-1,  1]])


In [5]:
x = torch.tensor(1)
print(x.shape)

x = torch.tensor([1, -1])
print(x.shape)

x = torch.tensor([[1, -1], [-1, 1]])
print(x.shape)

x = torch.tensor([[[1, 2, 3], [2 ,3, 1], [3, 1, 2]]])
print(x.shape)

torch.Size([])
torch.Size([2])
torch.Size([2, 2])
torch.Size([1, 3, 3])


In [6]:
x = torch.tensor([[[1, 2, 3],
                  [2, 3, 1],
                  [3, 1, 2],
                  [1, 2, 3]]])
x.shape

torch.Size([1, 4, 3])

In [7]:
x_zero = torch.zeros(2, 2)
x_one = torch.ones(2, 2)
x_eye = torch.eye(2)
display(x_zero, x_one, x_eye)

tensor([[0., 0.],
        [0., 0.]])

tensor([[1., 1.],
        [1., 1.]])

tensor([[1., 0.],
        [0., 1.]])

In [8]:
x = torch.tensor([[1, 2],
                  [2, 1]])
y = torch.tensor([[3, 4],
                  [4, 3]])

print(x + y)
print(x - y)
print(x * y)
print(x / y)

tensor([[4, 6],
        [6, 4]])
tensor([[-2, -2],
        [-2, -2]])
tensor([[3, 8],
        [8, 3]])
tensor([[0.3333, 0.5000],
        [0.5000, 0.3333]])


In [9]:
x = torch.tensor([1., 3., 5., 7., 9.])
print(x.sum())
print(x.mean())

tensor(25.)
tensor(5.)


In [10]:
x = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

x_transpose = x.T
x_reshape = x.reshape(3, 2)
display(x, x_transpose, x_reshape)

tensor([[1, 2, 3],
        [4, 5, 6]])

tensor([[1, 4],
        [2, 5],
        [3, 6]])

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [11]:
x = torch.tensor([[[1, 2, 3],
                  [4, 5, 6]]])

x_squeeze = x.squeeze(0)
x_unsqueeze_1 = x.unsqueeze(2)
x_unsqueeze_2 = x.unsqueeze(3)
display(x.shape, x_squeeze.shape, x_unsqueeze_1.shape, x_unsqueeze_2.shape)


torch.Size([1, 2, 3])

torch.Size([2, 3])

torch.Size([1, 2, 1, 3])

torch.Size([1, 2, 3, 1])

In [12]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [15]:
from torch.utils.data import Dataset, DataLoader, random_split


class IrisDataset(Dataset):
    def __init__(self, iris):
        self.x_data = torch.tensor(iris.data, dtype=torch.float32)
        self.y_data = torch.tensor(iris.target, dtype=torch.long)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.x_data)

In [16]:
import torch.nn as nn

class SoftmaxRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

In [17]:
iris = IrisDataset(iris)

In [18]:
train_size = int(len(iris) * 0.7)
valid_size = int(len(iris) - train_size)
train_dataset, valid_dataset = random_split(iris, [train_size, valid_size])

In [19]:
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=16, shuffle=False)

In [21]:
model = SoftmaxRegressionModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [22]:
epochs = 100
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(epochs):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        outputs = model (inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)
    train_accuracy = correct_train / total_train
    train_accuracies.append(train_accuracy)

    model.eval()
    running_valid_loss = 0.0
    correct_valid = 0
    total_valid = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model (inputs)
            loss = criterion(outputs, labels)
            running_valid_loss += loss. item()
            _, predicted = torch.max(outputs.data, 1)
            total_valid += labels.size(0)
            correct_valid += (predicted == labels).sum().item()

    epoch_valid_loss = running_valid_loss / len(valid_loader)
    valid_losses.append(epoch_valid_loss)
    valid_accuracy = correct_valid / total_valid
    valid_accuracies.append(valid_accuracy)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:,.4f}, Valid Loss: {epoch_valid_loss:,.4f}, '
              f'Train Acc: {train_accuracy:,.4f}, Valid Acc: {valid_accuracy:,.4f}')

Epoch [10/100], Train Loss: 0.7856, Valid Loss: 0.7611, Train Acc: 0.7048, Valid Acc: 0.7556
Epoch [20/100], Train Loss: 0.6547, Valid Loss: 0.6199, Train Acc: 0.8762, Valid Acc: 0.8667
Epoch [30/100], Train Loss: 0.5881, Valid Loss: 0.5442, Train Acc: 0.8571, Valid Acc: 0.8889
Epoch [40/100], Train Loss: 0.5411, Valid Loss: 0.4985, Train Acc: 0.8190, Valid Acc: 0.8889
Epoch [50/100], Train Loss: 0.5031, Valid Loss: 0.4616, Train Acc: 0.9143, Valid Acc: 0.8889
Epoch [60/100], Train Loss: 0.4894, Valid Loss: 0.4557, Train Acc: 0.9048, Valid Acc: 0.8667
Epoch [70/100], Train Loss: 0.4615, Valid Loss: 0.4215, Train Acc: 0.9143, Valid Acc: 0.9556
Epoch [80/100], Train Loss: 0.4369, Valid Loss: 0.4036, Train Acc: 0.9429, Valid Acc: 0.9556
Epoch [90/100], Train Loss: 0.4311, Valid Loss: 0.3874, Train Acc: 0.9524, Valid Acc: 0.9556
Epoch [100/100], Train Loss: 0.4157, Valid Loss: 0.3790, Train Acc: 0.9333, Valid Acc: 0.9556


In [24]:
model.eval()
all_preds, all_targets, all_probs = [], [], []
with torch.no_grad():
    for xb, yb in valid_loader:
        logits = model (xb)
        probs = torch.softmax(logits, dim=1)
        pred = probs.argmax(dim=1)
        all_preds.append(pred.cpu())
        all_targets.append(yb.cpu())
        all_probs.append(probs.cpu())
all_preds = torch.cat(all_preds).numpy()
all_targets = torch.cat(all_targets).numpy()
all_probs = torch.cat(all_probs).numpy()

names = load_iris().target_names

print ("\n === Validation Predictions === ")
for i in range(len(all_targets)):
    print(f"{i:03d} | true={names[all_targets[i]] :>10} "
        f"pred={names[all_preds[i]] :>10} "
        f"probs={np.round(all_probs[i], 3)}")


 === Validation Predictions === 
000 | true=    setosa pred=    setosa probs=[0.964 0.035 0.001]
001 | true=    setosa pred=    setosa probs=[0.913 0.082 0.005]
002 | true=    setosa pred=    setosa probs=[0.864 0.122 0.014]
003 | true=versicolor pred=versicolor probs=[0.072 0.536 0.391]
004 | true= virginica pred= virginica probs=[0.005 0.422 0.573]
005 | true=versicolor pred=versicolor probs=[0.102 0.632 0.266]
006 | true= virginica pred= virginica probs=[0.015 0.433 0.552]
007 | true= virginica pred=versicolor probs=[0.012 0.54  0.449]
008 | true= virginica pred= virginica probs=[0.011 0.322 0.667]
009 | true=versicolor pred=versicolor probs=[0.073 0.541 0.386]
010 | true= virginica pred= virginica probs=[0.006 0.243 0.751]
011 | true=versicolor pred=versicolor probs=[0.182 0.607 0.21 ]
012 | true=    setosa pred=    setosa probs=[0.892 0.101 0.008]
013 | true= virginica pred= virginica probs=[0.024 0.43  0.546]
014 | true=versicolor pred=versicolor probs=[0.052 0.628 0.319]
015 | 